compare results from 8A and irf cca results (6A)

07/22/2020


In [57]:
import os, glob
import pandas as pd
import seaborn as sns
import matplotlib as plt

In [58]:
normal_tissues = ['Airway','Astrocytes','Bladder','Colon','Esophageal','GDSD6','GM12878','HMEC','Melanocytes','Ovarian',
'Pancreas','Prostate','Renal','Thyroid','Uterine']

In [59]:
save_dir = '../data/processed/fig4_modelling/tf_tf_pairs_stability/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [66]:
# tf tf enrichment results
enrichment_path =  '../data/processed/fig4_modelling/tf_tf_pairs/'
sorted(glob.glob(os.path.join(enrichment_path,'*info.csv')))

['../data/processed/fig4_modelling/tf_tf_pairs/expr_Airway_loop_loop_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Airway_pro_pro_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Astrocytes_loop_loop_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Astrocytes_pro_pro_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Bladder_loop_loop_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Bladder_pro_pro_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Colon_loop_loop_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Colon_pro_pro_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Esophageal_loop_loop_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Esophageal_pro_pro_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_GDSD6_loop_loop_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/ex

In [67]:
# stability_path = '../data/processed/fig4_modelling/irf_manual/'

stability_path = '../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/'
sorted(glob.glob(os.path.join(stability_path, '*boosted_stability_score.csv')))

['../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/expr_Colon_Colon_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/expr_GDSD6_GDSD6_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/expr_GM12878_GM12878_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/expr_Melanocytes_Melanocytes_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/test1_GDSD6_GDSD6_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/test_GDSD6_GDSD6_boosted_stability_score.csv']

In [49]:
group_dict={'Airway':'purple','Astrocytes':'blue','Bladder':'purple','Colon':'green','Esophageal':'green','GDSD6':'purple','GM12878':'grey','HMEC':'purple','Melanocytes':'blue','Ovarian':'green',
'Pancreas':'green','Prostate':'purple','Renal':'green','Thyroid':'green','Uterine':'purple'}
    


In [50]:
def get_vocab_pair(vocab):
    vocab_arr = vocab.split('::')
    vocab_arr_trim = [x.split('_')[0] for x in vocab_arr]
    return '::'.join(vocab_arr_trim)


In [51]:
def merge_methods(tissue, save=None,loop_type='pro_pro', expr=True,group=False):
    print('++++++++',tissue, 'group', group, 'expr',expr, 'loop_type', loop_type)
    if expr:
        expr_file = os.path.join(enrichment_path, 'expr_'+tissue+'_' + loop_type + '_vocab_info.csv')
    else:
        expr_file = os.path.join(enrichment_path, 'unique_alltissue_'+loop_type+'_vocab_info.csv')
    if not os.path.isfile(expr_file):
        print('expr file not found, ', expr_file)
        return None
    else:
        print('loaded expr file: ', expr_file)
    df_enrich = pd.read_csv(expr_file, index_col=0)
    # df_enrich = pd.read_csv('../data/processed/fig4_modelling/tf_tf_pairs/unique_alltissue_vocab_info.csv', index_col=0)
    df_enrich = df_enrich[df_enrich.tissue==tissue]
    df_enrich_vocab = df_enrich.vocab.unique()
    print('num vocab enrichment', df_enrich_vocab.shape)
    
    if group:
        stability_file = '../data/processed/fig4_modelling/irf_manual/test_'+tissue+'_'+group_dict[tissue]+'_boosted_stability_score.csv'
    else:
        stability_file = '../data/processed/fig4_modelling/irf_manual/test_'+tissue+'_'+tissue+'_boosted_stability_score.csv'
    if not os.path.isfile(stability_file):
        print('stability file not exist', stability_file)
        return None 
    else:
        print('loaded stability file', stability_file)
    df_stability = pd.read_csv(stability_file)
    df_stability['vocab'] = df_stability['index'].apply(get_vocab_pair)

    df_enrich_stable =  df_enrich.merge(df_stability, how='inner',on='vocab',suffixes=('_tf','_model'))
    print('df_enrich_stable', df_enrich_stable.shape)
    if df_enrich_stable.shape[0]==0:
        return
    if save is not None:
        print('saving',tissue)
        if expr:
            if group:
                df_enrich_stable.to_csv(save+ tissue+'_'+loop_type+'_vocab_group.csv')            
            else:
                df_enrich_stable.to_csv(save+ tissue+'_'+loop_type+'_vocab.csv')
        else:
            if group:
                df_enrich_stable.to_csv(save+ tissue+'_'+loop_type+'_vocab_group_unique.csv')            
            else:
                df_enrich_stable.to_csv(save+ tissue+'_'+loop_type+'_vocab_unique.csv')


In [43]:
for tissue in normal_tissues:
    merge_methods(tissue,save=save_dir+'/', expr=True,group=False, loop_type= 'pro_pro')
    merge_methods(tissue,save=save_dir+'/',expr=True, group=True,loop_type= 'pro_pro')# unique to not tissue
    merge_methods(tissue,save=save_dir+'/',expr=False,group=False,loop_type= 'pro_pro') #unique to only tissue
    merge_methods(tissue,save=save_dir+'/', expr=True,group=False, loop_type= 'loop_loop')
    merge_methods(tissue,save=save_dir+'/',expr=True, group=True,loop_type= 'loop_loop')# unique to not tissue
    merge_methods(tissue,save=save_dir+'/',expr=False,group=False,loop_type= 'loop_loop') #unique to only tissue
        

++++++++ Airway group False expr True loop_type pro_pro
loaded expr file:  ../data/processed/fig4_modelling/tf_tf_pairs/expr_Airway_pro_pro_vocab_info.csv
num vocab enrichment (267,)
stability file not exist ../data/processed/fig4_modelling/irf_manual/test_Airway_Airway_boosted_stability_score.csv
++++++++ Airway group True expr True loop_type pro_pro
loaded expr file:  ../data/processed/fig4_modelling/tf_tf_pairs/expr_Airway_pro_pro_vocab_info.csv
num vocab enrichment (267,)
loaded stability file ../data/processed/fig4_modelling/irf_manual/test_Airway_purple_boosted_stability_score.csv
df_enrich_stable (8, 18)
saving Airway
++++++++ Airway group False expr False loop_type pro_pro
loaded expr file:  ../data/processed/fig4_modelling/tf_tf_pairs/unique_alltissue_pro_pro_vocab_info.csv
num vocab enrichment (0,)
stability file not exist ../data/processed/fig4_modelling/irf_manual/test_Airway_Airway_boosted_stability_score.csv
++++++++ Airway group False expr True loop_type loop_loop
loaded

In [52]:
for tissue in ['Melanocytes']:
    merge_methods(tissue,save=save_dir+'/', expr=True,group=False, loop_type= 'pro_pro')
    merge_methods(tissue,save=save_dir+'/',expr=True, group=True,loop_type= 'pro_pro')# unique to not tissue
    merge_methods(tissue,save=save_dir+'/',expr=False,group=False,loop_type= 'pro_pro') #unique to only tissue
    merge_methods(tissue,save=save_dir+'/', expr=True,group=False, loop_type= 'loop_loop')
    merge_methods(tissue,save=save_dir+'/',expr=True, group=True,loop_type= 'loop_loop')# unique to not tissue
    merge_methods(tissue,save=save_dir+'/',expr=False,group=False,loop_type= 'loop_loop') #unique to only tissue
        

++++++++ Melanocytes group False expr True loop_type pro_pro
loaded expr file:  ../data/processed/fig4_modelling/tf_tf_pairs/expr_Melanocytes_pro_pro_vocab_info.csv
num vocab enrichment (583,)
loaded stability file ../data/processed/fig4_modelling/irf_manual/test_Melanocytes_Melanocytes_boosted_stability_score.csv
df_enrich_stable (0, 18)
++++++++ Melanocytes group True expr True loop_type pro_pro
loaded expr file:  ../data/processed/fig4_modelling/tf_tf_pairs/expr_Melanocytes_pro_pro_vocab_info.csv
num vocab enrichment (583,)
loaded stability file ../data/processed/fig4_modelling/irf_manual/test_Melanocytes_blue_boosted_stability_score.csv
df_enrich_stable (0, 18)
++++++++ Melanocytes group False expr False loop_type pro_pro
loaded expr file:  ../data/processed/fig4_modelling/tf_tf_pairs/unique_alltissue_pro_pro_vocab_info.csv
num vocab enrichment (23,)
loaded stability file ../data/processed/fig4_modelling/irf_manual/test_Melanocytes_Melanocytes_boosted_stability_score.csv
df_enrich_

In [ ]:

# df_stability = pd.read_csv('../data/processed/fig4_modelling/irf_manual/test_GDSD6_GDSD6_boosted_stability_score.csv')
# df_stability['vocab'] = df_stability['index'].apply(get_vocab_pair)

# df_enrich.merge(df_stability, how='inner',on='vocab',suffixes=('_tf','_model'))

In [14]:
for tissue in normal_tissues:
    

,score,std,count
index,,,
AHR_pro,0.062759,0.052384,3
AHR_pro::ARNT_pro::BHLHE41_pro::DDIT3_pro::NR4A2_pro,0.034340,NaN,1
AHR_pro::ARNT_pro::CEBPB_pro,0.034340,NaN,1
AHR_pro::ARNT_pro::DDIT3_pro::NR4A2_pro::NRF1_pro,0.034340,NaN,1
AHR_pro::ARNT_pro::E2F7_pro::JUN_pro::TFDP1_pro,0.030787,NaN,1
...,...,...,...
ZNF524_pro::ZNF770_pro,0.035524,NaN,1
ZNF740_pro,0.088810,NaN,1
ZNF768_pro,0.035524,NaN,1
